# tensorflow 모델을 tensorflow-lite 모델로 변환하기
- https://www.tensorflow.org/lite/guide/get_started
    > 공식문서
- https://talkingaboutme.tistory.com/entry/Embedded-DL-Tensorflow-Lite-Example 
    > 코드 참고
- https://developers-kr.googleblog.com/2020/05/devsummit-2020-tensorflow-lite.html
    > DevSummit 2020에서 소개된 TensorFlow Lite의 새로운 기능
- 

In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [5]:
import tensorflow_model_optimization as tfmot

x = [-1,0,1,2,3,4]
y = [-3,-1,1,3,5,7]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])
# 양자화의 성능 및 크기 이점을 가진 모델 만들기
model.quantized_model = tfmot.quantization.keras.quantize_model(model)


model.compile(optimizer='adam', loss='mse')
model.fit(x,y, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 0s/step - loss: 12.5206
Epoch 2/10
1/1 [==============================] - 0s 998us/step - loss: 12.5015
Epoch 3/10
1/1 [==============================] - 0s 998us/step - loss: 12.4824
Epoch 4/10
1/1 [==============================] - 0s 999us/step - loss: 12.4633
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 12.4442
Epoch 6/10
1/1 [==============================] - 0s 0s/step - loss: 12.4251
Epoch 7/10
1/1 [==============================] - 0s 0s/step - loss: 12.4060
Epoch 8/10
1/1 [==============================] - 0s 997us/step - loss: 12.3870
Epoch 9/10
1/1 [==============================] - 0s 996us/step - loss: 12.3680
Epoch 10/10
1/1 [==============================] - 0s 997us/step - loss: 12.3490


In [6]:
## tlite용 모델로 convert 하는 과정
import pathlib

#save model to export_dir
export_dir = '.'
tf.saved_model.save(model, export_dir)

#convert the model to tflite model format
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

#Save the model
tflite_model_file = pathlib.Path('./a.tflite')
tflite_model_file.write_bytes(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: .\assets


776

In [25]:
# Run inference with the model

interpreter = tf.lite.Interpreter(model_path='./a.tflite')
interpreter.allocate_tensors() # 텐서 초기화

#giet input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details[0]['shape']) #[1 2]
print(input_details[0]['index']) #1
print(output_details[0]['index']) #0

x_test = np.array([5])
x_test = x_test.astype(dtype=np.float32)
interpreter.set_tensor(1, x_test)
#interpreter.set_tensor(input_details[0]['index'], x_test)

#inference 실행
interpreter.invoke()
y_predict = interpreter.get_tensor(0)
y_predict


[1 1]
0
3


array([[-5.2559553e-28]], dtype=float32)